<a href="https://colab.research.google.com/github/Yuns-u/Naver_Series_Reviews_Analysis/blob/main/testrank_naverseries_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표
한국어 Glove 모델을 이용하여 문장들을 임베딩 한 뒤, 임베딩 결과로 RANK를 메겨 상위 3개의 문장을 문서의 추출요약으로 선정.

In [ ]:
pip install konlpy

In [ ]:
import re
import konlpy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 데이터 불러오기

In [ ]:
#데이터 불러오기
from google.colab import files
uploaded = files.upload()


In [ ]:
df = pd.read_csv('IOS_ReviewData_naverseries_2.csv', engine='python')

In [ ]:
df

# 전처리: 한글 이외의 corpus 제거

In [ ]:
def cleaning(text): 
  repl ='' 
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거 
  text = re.sub(pattern= pattern, repl=repl, string=text) 
  #pattern = '[^\w\s]' # 특수기호 제거 
  pattern = '[^가-히\s]' # 특수기호 제거 
  text = re.sub(pattern= pattern, repl=repl, string=text) 
  pattern = '<[^>]*>' # html 제거 
  text = re.sub(pattern = pattern, repl='',string=text) 
  return text

In [ ]:
df['Reviews'] = df['Reviews'].map(lambda x: cleaning(x))
df

In [ ]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','요', '아니',
           '해서','에서','제','신건','아니','저기','주세요','해','서','되는','논','여','저','로','으로','오','고','랑','이랑',
           '합니다','부터','\n','\n\n','왜','입니다','에도','하고','만','다','너무','했는데','수','것','때','거','하는','있는',
           '하는']

In [ ]:
from konlpy.tag import Okt

tokenizer = Okt()

tokenized=[]
for sentence in df['Reviews']:
    temp = tokenizer.nouns(sentence) # 토큰화morphs를 쓸 때보다 명사만 추출했을 때 더 잘 파악됨. 한국어 표제어 추출이 어려움.
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)

print(tokenized[:10])

In [ ]:
df['Tokens'] = tokenized
df

In [ ]:
# 리뷰 길이 분포 확인
print('키워드의 최대 길이 :',max(len(l) for l in tokenized))
print('키워드의 평균 길이 :',sum(map(len, tokenized))/len(tokenized))
plt.hist([len(s) for s in tokenized], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

#전처리된 텍스트들의 길이 분포 살펴보기

In [ ]:
text_len = [len(s.split()) for s in df['Reviews']]
keys_len = [len(s) for s in df['Tokens']]

In [ ]:
print('원문의 최소 길이: ', np.min(text_len))
print('원문의 최대 길이: ', np.max(text_len))
print('원문의 평균 길이: ', np.mean(text_len))
print('키워드의 최소 길이: ', np.min(keys_len))
print('키워드의 최대 길이: ', np.max(keys_len))
print('키워드의 평균 길이: ', np.mean(keys_len))

In [ ]:
# 박스플롯으로 살펴보기

# 요약문 텍스트 길이 분포 박스플롯
plt.subplot(1,2,1)
plt.boxplot(keys_len)
plt.title('Text Length of keynouns')
plt.show()

# 원문 텍스트 길이 분포 박스플롯
plt.subplot(1,2,1)
plt.boxplot(text_len)
plt.title('Text Length of Original Text')
plt.show()

In [ ]:
# 히스토그램으로 살펴보기

# 요약문 텍스트 길이 분포 히스토그램
plt.title('Text Length of keynouns')
plt.hist(keys_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('the number of samples')
plt.show()

# 원문 텍스트 길이 분포 히스토그램
plt.title('Text Length of Original Text')
plt.hist(text_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('the number of samples')
plt.show()

원문 텍스트는 대체적으로 25이하의 길이를 가지며 평균적으로 13의 길이를 가지고 있다.


키워드의 경우 대체적으로 20이하의 길이를 가지며 평균 길이는 13이다. 

여기에서 패딩의 길이는 평균 길이정도인 25와 13으로 잡아주기로 했다.

In [ ]:
text_max_len =25
keys_max_len =13

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if (len(s.split()) <= max_len):
      cnt = cnt+1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt/len(nested_list))))

In [ ]:
below_threshold_len(text_max_len, df['Reviews'])

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if (len(s) <= max_len):
      cnt = cnt+1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt/len(nested_list))))

In [ ]:
below_threshold_len(keys_max_len, df['Tokens'])

 정해준 최대 길이보다 큰 샘플들은 연산의 수월성을 위하여 제거하고자 한다.

In [ ]:
df = df[df['Reviews'].apply(lambda x: len(x.split()) <= text_max_len)]
df = df[df['Tokens'].apply(lambda x: len(x) <= keys_max_len)]
df

In [ ]:
#줄면서 없어진 것은 결측치로 없애주기
df['Tokens'].describe()

In [ ]:
print('전체 데이터의 수: ',len(df))

In [ ]:
test_num = int(len(df)*0.2)
test_num
#전체데이터의 20%는 64개라고 할 수 있다.

In [ ]:
# train, test data 분류 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Reviews']], df['Tokens'], train_size = 0.8, test_size=0.2, random_state=2)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
y_train

-----------

In [ ]:
df['Reviews'].to_json('ios_reviews.json',orient='columns',force_ascii=False)
#로컬환경으로 다운로드받기
from google.colab import files
files.download('ios_reviews.json')

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
import json

with open('ios_reviews.json', 'r', encoding='utf-8-sig') as f:
    comment = json.load(f)

In [ ]:
review_list = np.array(df['Reviews'].tolist())
review_list[:3]

In [ ]:
key_sentence = {}

for res in review_list:
    review = comment[res]
    su = summarize(review, word_count=5)
    su = re.sub('\n', ' ',su)
    if len(su) == 0:
        continue
    key_sentence[res] = su

----------

In [ ]:
#토큰화된 문서들을 입력받아 토큰을 센 뒤 관련된 속성을 가진 데이터프레임을 return
def word_count(docs):
  #전체 코퍼스에서 단어 빈도 카운트
  word_counts = Counter()

  #단어가 존재하는 문서의 빈도 카운트, 단어가 한 번 이상 존재하면 1을 더한다.
  word_in_docs = Counter()

  #전체 문서의 개수
  total_docs = len(docs)

  for doc in docs:
    word_counts.update(doc)
    word_in_docs.update(set(doc))

  temp = zip(word_counts.keys(), word_counts.values())
  wc = pd.DataFrame(temp, columns=['word','count'])

  #단어 순위
  #method='first':같은 값의 경우 먼저 나온 요소를 우선
  wc['rank'] = wc['count'].rank(method='first', ascending=False)
  total = wc['count'].sum()

  #코퍼스 내 단어의 비율
  wc['percent'] = wc['count'].apply(lambda x: x/ total)
  wc = wc.sort_values(by='rank')

  #누적 비율
  #cumsum() : cumulative sum
  wc['cul_percent'] = wc['percent'].cumsum()

  temp2 = zip(word_in_docs.keys(), word_in_docs.values())
  ac = pd.DataFrame(temp2, columns=['word','word_in_docs'])
  wc = ac.merge(wc, on='word')

  #전체 문서 중 존재하는 비율
  wc['word_in_docs_percent'] = wc['word_in_docs'].apply(lambda x: x/total_docs)

  return wc.sort_values(by='rank')

In [ ]:
wc = word_count(df['Tokens'])
wc.head(20)

In [ ]:
wc.describe()

In [ ]:
#토큰 순위에 따른 퍼센트 누적 분포 그래프
import seaborn as sns

sns.lineplot(x='rank', y='cul_percent', data=wc)

# ----------

In [ ]:
pip install gensim --upgrade

In [ ]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

In [ ]:
model = Word2Vec(sentences = tokenized, window = 3, min_count = 2, workers = 4, sg = 0)

In [ ]:
model.wv.vectors.shape

In [ ]:
model.wv.most_similar("쿠키")

In [ ]:
model.wv.most_similar("어디")